In [1]:
import numpy as np
import cv2 
import matplotlib.pylab
import io
import os

In [2]:
def augmented_image(frame,im_src, pts_src, pts_dst):
    
    # Calculate Homography
    h, status = cv2.findHomography(pts_src, pts_dst)

    # Warp source image to destination based on homography
    warped_image = cv2.warpPerspective(im_src, h, (frame.shape[1],frame.shape[0]))
            
    # Prepare a mask representing region to copy from the warped image into the original frame.
    mask = np.zeros([frame.shape[0], frame.shape[1]], dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.int32(pts_dst), (255, 255, 255), cv2.LINE_AA)
    im_out = cv2.add(frame, warped_image, mask=cv2.bitwise_not(mask))
    im_out = cv2.add(im_out, warped_image)
    
    return im_out

In [3]:
params_dir = os.getcwd()+'/camera_params/monocular_camera_params/'

#load camera parameters
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')

In [4]:
AruCo_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_1000)
AruCo_params = cv2.aruco.DetectorParameters_create()
board = cv2.aruco.GridBoard_create(3, 4, 0.05, 0.0075, AruCo_dict)

In [5]:
def draw_axis(img, corners, imgpts) :
    corner = tuple(corners[0].ravel().astype(int))
    img = cv2.line(img, corner, tuple(imgpts[0].ravel().astype(int)), (255,0,0), 10)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel().astype(int)), (0,255,0), 10)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel().astype(int)), (0,0,255), 10)
    return img

In [6]:
def write_text(img, pose, dy, text) :
    x0 = pose[0]
    y0 = pose[1]
    for i, line in enumerate(text.split('\n')) :
        y = y0 + i*dy
        cv2.putText(img, line, np.int32([x0, y]), cv2.FONT_HERSHEY_COMPLEX, 0.75, (50,200,255), 2)

In [9]:
vid = cv2.VideoCapture('./Forder Final/left_output-1.avi') #การเปิดวีดีโอหนังสือเคลื่อนที่
template_img = cv2.imread('./Forder Final/Template-1.png') #การเปิดภาพหนังสือ
template_gray = cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)

sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

while vid.isOpened() :
    ret, frame = vid.read()
    im_src = cv2.imread('./images/newqr.jpg') #ภาพที่จะนำมาแทรกภาพตรงกลาง 
    im_src_size = im_src.shape[:2]
    src_points = np.float32([[0,0], [im_src_size[1],0],[im_src_size[1], im_src_size[0]] ,[0, im_src_size[0]] ])
    
    if ret :
        query = frame
        query_gray = cv2.cvtColor(query, cv2.COLOR_BGR2GRAY)

        template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
        query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
        matches = bf.knnMatch(template_desc, query_desc, k=2)
        good_matches = list()
        good_matches_list = list()
        for m, n in matches :
            if m.distance < 0.7*n.distance :
                good_matches.append(m)
                good_matches_list.append([m])
        
        if len(good_matches) > 10 :
            src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
            dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

            H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 10.0) # H RANSAC
            # get the bounding box around template image
            h, w = template_img.shape[:2]
            template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
            transformed_box = cv2.perspectiveTransform(template_box, H)

            detected_img = cv2.polylines(query, [np.int32(transformed_box)], True, (255,0,0), 3, cv2.LINE_AA)
            drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)

            augmented = augmented_image(detected_img, im_src, src_points, transformed_box) #src หามุม

            img = augmented.copy()
        
            markerCorners, markerIds, rejectedCandidates = cv2.aruco.detectMarkers( augmented, AruCo_dict, parameters = AruCo_params)
            
            if len(markerCorners) > 0:
                img = cv2.aruco.drawDetectedMarkers(augmented, markerCorners)
                rvecs, tvecs, points = cv2.aruco.estimatePoseSingleMarkers(markerCorners , 0.05, K, dist)
                for (rvec, tvec, id, corner) in zip(rvecs, tvecs, markerIds, markerCorners) :
                    img = cv2.aruco.drawAxis(augmented, K, dist, rvec, tvec, 0.05)
                    x = tvec[0,0]
                    y = tvec[0,1]
                    z = tvec[0,2]
                    text = "id: {}\n pose:\n {:.3f}\n {:.3f}\n {:.3f}".format(id, x, y, z)
                    cX = (corner[0,0][0] + corner[0,2][0]) / 2
                    cY = (corner[0,0][1] + corner[0,2][1]) / 2
                    write_text(detected_img, (cX, cY), 20, text)
                ret, brvec, btvec = cv2.aruco.estimatePoseBoard(markerCorners, markerIds, board, K, dist, rvecs, tvecs) 
                if ret :
                    img = cv2.aruco.drawAxis(detected_img, K, dist, brvec, btvec, 0.05)

        cv2.imshow('Video frame',detected_img)

        if cv2.waitKey(int(1000/24)) & 0xFF == ord('q') : # this line control the period between image frame
            break
    else :
        break
vid.release()
cv2.destroyAllWindows()